In [122]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from pathlib import Path
import torch.nn as nn
import warnings
import torch.optim as optim
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report
from tqdm import tqdm
import time
import numpy as np
from PIL import Image
from collections import Counter
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import random
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [123]:
# Settings
train_dir = Path('./data/train')
val_dir   = Path('./data/test')
img_size = 48
batch_size = 64

# Transforms
train_transforms = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(5),
    transforms.RandomAffine(degrees=0, shear=10, translate=(0.2, 0.2)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)  # normalize to [-1, 1]
])

val_transforms = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])

# Datasets
train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transforms)
val_dataset   = datasets.ImageFolder(root=val_dir, transform=val_transforms)
print(f"Train images: {len(train_dataset)}")
print(f"Val images: {len(val_dataset)}")

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

# Class names
class_names = train_dataset.classes
print("Classes:", class_names)

Train images: 33595
Val images: 7178
Classes: ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


In [124]:
import torch.nn as nn
import torch.nn.functional as F

class ResSim(nn.Module):
    def __init__(self, num_classes=7, dropout_rate=0.15):
        super(ResSim, self).__init__()
        self.dropout = nn.Dropout(dropout_rate)

        # Block 1
        self.block1 = nn.Sequential(
            nn.Conv2d(3, 24, kernel_size=3, padding=1),
            nn.BatchNorm2d(24),
            nn.ReLU(),
            nn.Conv2d(24, 24, kernel_size=3, padding=1),
            nn.BatchNorm2d(24)
        )
        self.shortcut1 = nn.Conv2d(3, 24, kernel_size=1)

        # Block 2
        self.block2 = nn.Sequential(
            nn.Conv2d(24, 48, kernel_size=3, padding=1),
            nn.BatchNorm2d(48),
            nn.ReLU(),
            nn.Conv2d(48, 48, kernel_size=3, padding=1),
            nn.BatchNorm2d(48)
        )
        self.shortcut2 = nn.Conv2d(24, 48, kernel_size=1)

        self.pool = nn.MaxPool2d(2, 2)

        # Final FC layer stays at 64
        self.fc = nn.Sequential(
            nn.Linear(12 * 12 * 48, 64),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        x1 = self.block1(x)
        x = self.pool(F.relu(x1 + self.shortcut1(x)))
        x = self.dropout(x)

        x2 = self.block2(x)
        x = self.pool(F.relu(x2 + self.shortcut2(x)))
        x = self.dropout(x)

        x = x.view(x.size(0), -1)
        return self.fc(x)

In [125]:
model = ResSim(num_classes=7)
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params}")

Total parameters: 481551


In [126]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ResSim(num_classes=7).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [127]:
from sklearn.metrics import classification_report
import time
from tqdm import tqdm

def train(model, train_loader, val_loader, optimizer, criterion, num_epochs=15, model_name='resnet_model.pth'):
    history = []  # collect metrics per epoch

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        start_time = time.time()

        # Training phase
        model.train()
        train_loss, correct, total = 0, 0, 0
        for images, labels in tqdm(train_loader, desc="Training"):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        train_acc = correct / total
        train_loss /= total

        # Validation phase
        model.eval()
        val_loss, correct, total = 0, 0, 0
        y_true, y_pred = [], []

        with torch.no_grad():
            for images, labels in tqdm(val_loader, desc="Validation"):
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * images.size(0)

                _, predicted = outputs.max(1)
                correct += (predicted == labels).sum().item()
                total += labels.size(0)

                y_true.extend(labels.cpu().numpy())
                y_pred.extend(predicted.cpu().numpy())

        val_acc = correct / total
        val_loss /= total

        # Classification report
        report = classification_report(y_true, y_pred, target_names=class_names, output_dict=True)
        val_f1_macro = report['macro avg']['f1-score']

        epoch_time = time.time() - start_time

        # Collect metrics
        epoch_data = {
            'epoch': epoch + 1,
            'train_loss': train_loss,
            'val_loss': val_loss,
            'train_acc': train_acc,
            'val_acc': val_acc,
            'val_f1_macro': val_f1_macro,
            'epoch_time_sec': epoch_time
        }

        # Add per-class metrics
        for cls in class_names:
            for metric in ['precision', 'recall', 'f1-score']:
                key = f'{cls}_{metric}'
                epoch_data[key] = report[cls][metric]

        history.append(epoch_data)

        # Console output
        print("\nValidation Report:")
        for cls in class_names:
            cls_metrics = report[cls]
            print(f"{cls}: Prec={cls_metrics['precision']:.3f} | Rec={cls_metrics['recall']:.3f} | F1={cls_metrics['f1-score']:.3f}")

        print(f"\nEpoch Summary: Train Loss={train_loss:.4f} | Val Loss={val_loss:.4f} | Train Acc={train_acc:.3f} | Val Acc={val_acc:.3f}")
        print(f"Epoch time: {epoch_time:.2f} seconds")

    torch.save(model.state_dict(), f"results/{model_name}")
    print(f"Saved model to 'results/{model_name}'")

    return history

In [129]:
def build_model(dropout):
    return ResSim(num_classes=len(class_names), dropout_rate=dropout).to(device)

# Define settings to test
configs = [
    {'name': 'adam_lr3_dropout0_bn', 'optimizer': 'adam', 'lr': 1e-3, 'dropout': 0},
    {'name': 'adam_lr3_dropout15_bn', 'optimizer': 'adam', 'lr': 1e-3, 'dropout': 0.15},
    {'name': 'adam_lr2_dropout15_bn', 'optimizer': 'adam', 'lr': 1e-2, 'dropout': 0.15},
    {'name': 'adam_lr3_dropout20_bn', 'optimizer': 'adam', 'lr': 1e-3, 'dropout': 0.20},
    {'name': 'sgd_mom09_lr3_dropout15_bn', 'optimizer': 'sgd', 'lr': 1e-3, 'dropout': 0.15},
]


for config in configs:
    print(f"\nTraining config: {config['name']}")

    model = build_model(config['dropout'])
    if config['optimizer'] == 'adam':
        optimizer = optim.Adam(model.parameters(), lr=config['lr'])
    else:
        optimizer = optim.SGD(model.parameters(), lr=config['lr'], momentum=0.9)

    criterion = nn.CrossEntropyLoss()
    history = train(model, train_loader, val_loader, optimizer, criterion, num_epochs=15, model_name=config['name'])
    pd.DataFrame(history).to_csv(f'results/history_{config["name"]}.csv', index=False)


Training config: adam_lr3_dropout0_bn

Epoch 1/15


Validation: 100%|██████████| 113/113 [00:17<00:00,  6.41it/s]
/Users/yuyi/Desktop/DeepLearningProject/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/yuyi/Desktop/DeepLearningProject/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/yuyi/Desktop/DeepLearningProject/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_divisio


Validation Report:
angry: Prec=0.000 | Rec=0.000 | F1=0.000
disgust: Prec=0.005 | Rec=0.009 | F1=0.006
fear: Prec=0.176 | Rec=0.092 | F1=0.121
happy: Prec=0.375 | Rec=0.711 | F1=0.491
neutral: Prec=0.361 | Rec=0.011 | F1=0.020
sad: Prec=0.290 | Rec=0.200 | F1=0.236
surprise: Prec=0.294 | Rec=0.769 | F1=0.426

Epoch Summary: Train Loss=1.8889 | Val Loss=1.7705 | Train Acc=0.234 | Val Acc=0.314
Epoch time: 136.50 seconds

Epoch 2/15


Validation: 100%|██████████| 113/113 [00:18<00:00,  6.12it/s]
/Users/yuyi/Desktop/DeepLearningProject/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/yuyi/Desktop/DeepLearningProject/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/yuyi/Desktop/DeepLearningProject/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_divisio


Validation Report:
angry: Prec=0.000 | Rec=0.000 | F1=0.000
disgust: Prec=0.027 | Rec=0.144 | F1=0.045
fear: Prec=0.067 | Rec=0.002 | F1=0.004
happy: Prec=0.306 | Rec=0.889 | F1=0.456
neutral: Prec=0.000 | Rec=0.000 | F1=0.000
sad: Prec=0.278 | Rec=0.067 | F1=0.107
surprise: Prec=0.494 | Rec=0.655 | F1=0.563

Epoch Summary: Train Loss=1.6914 | Val Loss=1.7773 | Train Acc=0.340 | Val Acc=0.310
Epoch time: 119.90 seconds

Epoch 3/15


...


Validation Report:
angry: Prec=0.298 | Rec=0.140 | F1=0.190
disgust: Prec=0.000 | Rec=0.000 | F1=0.000
fear: Prec=0.267 | Rec=0.129 | F1=0.174
happy: Prec=0.449 | Rec=0.881 | F1=0.595
neutral: Prec=0.355 | Rec=0.436 | F1=0.391
sad: Prec=0.382 | Rec=0.156 | F1=0.222
surprise: Prec=0.648 | Rec=0.529 | F1=0.583

Epoch Summary: Train Loss=1.5114 | Val Loss=1.5351 | Train Acc=0.421 | Val Acc=0.418
Epoch time: 155.21 seconds

Epoch 14/15


Validation: 100%|██████████| 113/113 [00:17<00:00,  6.30it/s]



Validation Report:
angry: Prec=0.351 | Rec=0.071 | F1=0.118
disgust: Prec=0.030 | Rec=0.126 | F1=0.048
fear: Prec=0.268 | Rec=0.078 | F1=0.121
happy: Prec=0.457 | Rec=0.865 | F1=0.598
neutral: Prec=0.397 | Rec=0.392 | F1=0.394
sad: Prec=0.406 | Rec=0.195 | F1=0.263
surprise: Prec=0.521 | Rec=0.656 | F1=0.581

Epoch Summary: Train Loss=1.4960 | Val Loss=1.5822 | Train Acc=0.428 | Val Acc=0.413
Epoch time: 151.36 seconds

Epoch 15/15


Validation: 100%|██████████| 113/113 [00:17<00:00,  6.41it/s]


Validation Report:
angry: Prec=0.303 | Rec=0.151 | F1=0.202
disgust: Prec=0.022 | Rec=0.099 | F1=0.035
fear: Prec=0.234 | Rec=0.124 | F1=0.162
happy: Prec=0.538 | Rec=0.771 | F1=0.634
neutral: Prec=0.424 | Rec=0.376 | F1=0.399
sad: Prec=0.343 | Rec=0.298 | F1=0.319
surprise: Prec=0.574 | Rec=0.639 | F1=0.605

Epoch Summary: Train Loss=1.4685 | Val Loss=1.5332 | Train Acc=0.440 | Val Acc=0.420
Epoch time: 135.73 seconds
Saved model to 'results/sgd_mom09_lr3_dropout15_bn'


In [ ]:
mode = build_model(0.0)
metrics_history = train(model, train_loader, val_loader, optimizer, criterion, num_epochs=50, model_name='resnet_model_final.pth')
df_metrics = pd.DataFrame(metrics_history)
display(df_metrics.head())

# Small test

In [130]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
from collections import defaultdict
import numpy as np
import random

# Transform (with normalization to [-1, 1] range)
transform = transforms.Compose([
    transforms.Resize((48, 48)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # For 3 channels, use 3 values if needed
])

# Load full dataset
full_train_dataset = datasets.ImageFolder('./data/train', transform=transform)

# Limit to 500 samples per class
class_counts = defaultdict(list)
for idx, (_, label) in enumerate(full_train_dataset):
    if len(class_counts[label]) < 500:
        class_counts[label].append(idx)

# Flatten selected indices
subset_indices = [idx for indices in class_counts.values() for idx in indices]

# Create subset dataset
small_train_dataset = Subset(full_train_dataset, subset_indices)
train_loader = DataLoader(small_train_dataset, batch_size=32, shuffle=True)

# Validation loader (no subsample here, or you can do similarly)
val_dataset = datasets.ImageFolder('./data/test', transform=transform)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [131]:
metrics_history = train(model, train_loader, val_loader, optimizer, criterion, num_epochs=15)


Epoch 1/15


Validation: 100%|██████████| 225/225 [00:07<00:00, 30.87it/s]



Validation Report:
angry: Prec=0.276 | Rec=0.301 | F1=0.288
disgust: Prec=0.099 | Rec=0.450 | F1=0.162
fear: Prec=0.305 | Rec=0.031 | F1=0.057
happy: Prec=0.594 | Rec=0.807 | F1=0.684
neutral: Prec=0.406 | Rec=0.474 | F1=0.437
sad: Prec=0.444 | Rec=0.208 | F1=0.283
surprise: Prec=0.531 | Rec=0.697 | F1=0.603

Epoch Summary: Train Loss=1.6365 | Val Loss=1.4681 | Train Acc=0.371 | Val Acc=0.449
Epoch time: 18.90 seconds

Epoch 2/15


Validation: 100%|██████████| 225/225 [00:07<00:00, 30.56it/s]



Validation Report:
angry: Prec=0.386 | Rec=0.138 | F1=0.203
disgust: Prec=0.044 | Rec=0.784 | F1=0.083
fear: Prec=0.211 | Rec=0.154 | F1=0.178
happy: Prec=0.804 | Rec=0.556 | F1=0.657
neutral: Prec=0.497 | Rec=0.182 | F1=0.267
sad: Prec=0.346 | Rec=0.362 | F1=0.354
surprise: Prec=0.523 | Rec=0.702 | F1=0.599

Epoch Summary: Train Loss=1.5475 | Val Loss=1.6246 | Train Acc=0.413 | Val Acc=0.365
Epoch time: 19.04 seconds

Epoch 3/15


...


Validation Report:
angry: Prec=0.404 | Rec=0.177 | F1=0.247
disgust: Prec=0.241 | Rec=0.604 | F1=0.344
fear: Prec=0.312 | Rec=0.234 | F1=0.268
happy: Prec=0.702 | Rec=0.724 | F1=0.713
neutral: Prec=0.410 | Rec=0.531 | F1=0.463
sad: Prec=0.367 | Rec=0.496 | F1=0.422
surprise: Prec=0.749 | Rec=0.538 | F1=0.626

Epoch Summary: Train Loss=0.9664 | Val Loss=1.3812 | Train Acc=0.648 | Val Acc=0.485
Epoch time: 22.14 seconds

Epoch 14/15


Validation: 100%|██████████| 225/225 [00:09<00:00, 22.77it/s]



Validation Report:
angry: Prec=0.407 | Rec=0.234 | F1=0.297
disgust: Prec=0.202 | Rec=0.604 | F1=0.302
fear: Prec=0.284 | Rec=0.319 | F1=0.301
happy: Prec=0.711 | Rec=0.713 | F1=0.712
neutral: Prec=0.443 | Rec=0.501 | F1=0.470
sad: Prec=0.402 | Rec=0.354 | F1=0.376
surprise: Prec=0.625 | Rec=0.657 | F1=0.640

Epoch Summary: Train Loss=0.9668 | Val Loss=1.3780 | Train Acc=0.648 | Val Acc=0.486
Epoch time: 23.74 seconds

Epoch 15/15


Validation: 100%|██████████| 225/225 [00:08<00:00, 25.40it/s]



Validation Report:
angry: Prec=0.374 | Rec=0.219 | F1=0.276
disgust: Prec=0.152 | Rec=0.694 | F1=0.250
fear: Prec=0.278 | Rec=0.261 | F1=0.269
happy: Prec=0.715 | Rec=0.707 | F1=0.711
neutral: Prec=0.467 | Rec=0.463 | F1=0.465
sad: Prec=0.406 | Rec=0.354 | F1=0.378
surprise: Prec=0.559 | Rec=0.730 | F1=0.633

Epoch Summary: Train Loss=0.8985 | Val Loss=1.4449 | Train Acc=0.669 | Val Acc=0.478
Epoch time: 22.42 seconds
Saved model to 'results/resnet_model.pth'


In [132]:
df_metrics = pd.DataFrame(metrics_history)
display(df_metrics.head())

,epoch,train_loss,val_loss,train_acc,val_acc,val_f1_macro,epoch_time_sec,angry_precision,angry_recall,angry_f1-score,...,happy_f1-score,neutral_precision,neutral_recall,neutral_f1-score,sad_precision,sad_recall,sad_f1-score,surprise_precision,surprise_recall,surprise_f1-score
0,1,1.636541,1.468103,0.370571,0.449011,0.359029,18.902437,0.276392,0.300626,0.288000,...,0.683871,0.405556,0.473642,0.436962,0.444254,0.207698,0.283060,0.531193,0.696751,0.602811
1,2,1.547459,1.624593,0.412571,0.365283,0.334528,19.039200,0.385965,0.137787,0.203077,...,0.657333,0.496689,0.182482,0.266904,0.346390,0.361668,0.353864,0.523339,0.701564,0.599486
2,3,1.479716,1.430039,0.438000,0.453608,0.378196,21.285646,0.318725,0.167015,0.219178,...,0.701253,0.392105,0.483374,0.432982,0.364114,0.388933,0.376115,0.697640,0.569194,0.626905
3,4,1.413888,1.405892,0.456000,0.466704,0.404813,22.752694,0.342428,0.262004,0.296866,...,0.706971,0.410848,0.534469,0.464575,0.386809,0.348035,0.366399,0.550279,0.711191,0.620472
4,5,1.352093,1.355522,0.488286,0.486486,0.414913,22.773255,0.376923,0.255741,0.304726,...,0.699612,0.447518,0.446067,0.446791,0.366114,0.495589,0.421124,0.576699,0.714801,0.638366


In [133]:
train(model, train_loader, val_loader, num_epochs=15)

TypeError: train() missing 2 required positional arguments: 'optimizer' and 'criterion'

## Freeze Layers

In [ ]:
import torch.nn as nn
import torchvision.models as models

def build_light_resnet(num_classes):
    # Load pretrained resnet18
    base_model = models.resnet18(pretrained=True)

    # Freeze all layers
    for param in base_model.parameters():
        param.requires_grad = False

    # Replace the final classifier
    base_model.fc = nn.Sequential(
        nn.Linear(base_model.fc.in_features, 64),
        nn.BatchNorm1d(64),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(64, num_classes)
    )

    return base_model

In [ ]:
model = build_light_resnet(num_classes=len(class_names))

total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total Parameters: {total_params}")
print(f"Trainable Parameters: {trainable_params}")

/Users/yuyi/Desktop/DeepLearningProject/venv/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/yuyi/Desktop/DeepLearningProject/venv/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /Users/yuyi/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:06<00:00, 7.14MB/s]

Total Parameters: 11209927
Trainable Parameters: 33415
